# Solar_Irradiance_Prediction_using_Transformers

In [ ]:
#Imports
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import (mean_squared_error,
                             mean_absolute_error)
import time
import re
import os   
import torch, math
import numpy as np
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn.modules.transformer import TransformerEncoderLayer
import pickle
import base64

In [ ]:
# Get today's date and time(for saving file)
new_file_number="DEMO"
formatted_datetime = "XX_XX_XXXX"

In [ ]:
df_global=pd.read_csv("global.csv")
df_local=pd.read_csv("local.csv")

In [ ]:
# Create a figure and two subplots (1 column, 2 rows)
fig, axs = plt.subplots(2, 1, figsize=(15, 8))  # 2 rows, 1 column

# Plot the first subplot
axs[0].plot(df_global["shortwave_radiation_instant (W/m²)"], color='blue')
axs[0].set_title('Global Irradiance')
axs[0].set_ylabel('Irradiance')
axs[0].grid()

# Plot the second subplot
axs[1].plot(df_local["Solar Radiation (W/m^2)"], color='green')
axs[1].set_title('Local Irradiance')
axs[1].set_ylabel('Irradiance')
axs[1].set_xlabel('Time(15 MIN)')
axs[1].grid()

# # Adjust layout
# plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
#Gettign Index Number to Save File in Numerical Order
results_dir = "Results_Table"
saved_models_dir="saved_models"

existing_files = os.listdir(results_dir)
pattern = r"^(\d+)_Results"
existing_numbers = []

for filename in existing_files:
    match = re.match(pattern, filename)
    if match:
        existing_numbers.append(int(match.group(1)))

# If no existing files, start from 1
if existing_numbers:
    new_file_number = max(existing_numbers) + 1
else:
    new_file_number = 1



In [ ]:
#Forecoasting Model
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 10000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
    def forward(self, x: Tensor) -> Tensor:
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# A forcasting model
class ForecastingModel(torch.nn.Module):
    def __init__(self, 
                 seq_len,
                 embed_size = 8, 
                 nhead = 2,
                 dim_feedforward = 1024, 
                 dropout = 0.1,
                 conv1d_emb = True,
                 conv1d_kernel_size = 3,
                 device = "cuda"):
        super(ForecastingModel, self).__init__()

        # Set Class-level Parameters
        self.device = device
        self.conv1d_emb = conv1d_emb
        self.conv1d_kernel_size = conv1d_kernel_size
        self.seq_len = seq_len
        self.embed_size = embed_size

        # Input Embedding Component
        if conv1d_emb:
            if conv1d_kernel_size%2==0:
                raise Exception("conv1d_kernel_size must be an odd number to preserve dimensions.")
            self.conv1d_padding = conv1d_kernel_size - 1
            self.input_embedding  = nn.Conv1d(1, embed_size, kernel_size=conv1d_kernel_size)
        else: self.input_embedding  = nn.Linear(1, embed_size)

        # Positional Encoder Componet (See Code Copied from PyTorch Above)
        self.position_encoder = PositionalEncoding(d_model=embed_size, 
                                                   dropout=dropout,
                                                   max_len=seq_len)
        
        # Transformer Encoder Layer Component
        self.transformer_encoder = TransformerEncoderLayer(
            d_model = embed_size,
            nhead = nhead,
            dim_feedforward = dim_feedforward,
            dropout = dropout,
            batch_first = True
        )

        # Regression Component
        self.linear1 = nn.Linear(seq_len*embed_size, int(dim_feedforward))
        self.linear2 = nn.Linear(int(dim_feedforward), int(dim_feedforward/2))
        self.linear3 = nn.Linear(int(dim_feedforward/2), int(dim_feedforward/4))
        self.linear4 = nn.Linear(int(dim_feedforward/4), int(dim_feedforward/16))
        self.linear5 = nn.Linear(int(dim_feedforward/16), int(dim_feedforward/64))
        self.outlayer = nn.Linear(int(dim_feedforward/64), 1)

        # Basic Components
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    # Model Forward Pass
    def forward(self, x):
        src_mask = self._generate_square_subsequent_mask()
        src_mask.to(self.device)
        if self.conv1d_emb: 
            x = F.pad(x, (0, 0, self.conv1d_padding, 0), "constant", -1)
            x = self.input_embedding(x.transpose(1, 2))
            x = x.transpose(1, 2)
        else: 
            x = self.input_embedding(x)
        x = self.position_encoder(x)
        x = self.transformer_encoder(x, src_mask=src_mask).reshape((-1, self.seq_len*self.embed_size))
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear4(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear5(x)
        x = self.relu(x)
        return self.outlayer(x)
    
    # Function Copied from PyTorch Library to create upper-triangular source mask
    def _generate_square_subsequent_mask(self):
        return torch.triu(
            torch.full((self.seq_len, self.seq_len), float('-inf'), dtype=torch.float32, device=self.device),
            diagonal=1,
        )


In [ ]:
# Create a dataset

#Number of datapoints in 24 Hours
Hours_24=96 #96 for 15min Dataset Model (Should be 288 for 5min dataset model)

data_x = list(df_global['shortwave_radiation_instant (W/m²)'])
data_y = list(df_local["Solar Radiation (W/m^2)"])
x = np.array(data_x[:-Hours_24])  # Example size of x
y = np.array(data_y[:-Hours_24])  # Example size of y
forcast = np.array(data_y[-Hours_24:])  # Forecast data

#Training Loop GRID
seq_len = [24, 48, 96, 192, 240, 288, 384]
batch = [1, 2, 3, 4, 5, 6 ,7]
Epochs = [80, 90, 100, 200, 300, 400, 500]
learning_rate=[0.01,0.05,0.001,0.005,0.00001,0.00005,6.6E-6]


In [ ]:
# Define directories to save models and plots
model_save_dir = "saved_models"
plot_save_dir = "saved_plots"

In [ ]:
execution_time = []
results = [] 
models_csv=[]
run_number=1

#Training
for i in range(len(seq_len)):
    for j in range(len(batch)):  # Loop over batch sizes
        for k in range(len(Epochs)):  # Loop over epochs
            for m in range(len(learning_rate)):
                # Rearranging Data Shape
                X = np.array([x[ii:ii + seq_len[i]] for ii in range(0, x.shape[0] - seq_len[i])]).reshape((-1, seq_len[i], 1))
                Y = np.array([y[ii + seq_len[i]] for ii in range(0, y.shape[0] - seq_len[i])]).reshape((-1, 1))
    
                start_time = time.time()
                device = "cuda:0"
                EPOCHS = Epochs[k]  # Select the current epoch count
                BATCH_SIZE = batch[j]  # Select the current batch size
                LEARNING_RATE = learning_rate[m]

                model = ForecastingModel(seq_len=seq_len[i], embed_size=8, nhead=2, 
                                         dim_feedforward=1024, dropout=0.1, 
                                         conv1d_emb=True, conv1d_kernel_size=3, device=device).to(device)
                
                
                model.train()
                criterion = torch.nn.HuberLoss()
                optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate[m])
                scheduler = ExponentialLR(optimizer, gamma=0.9)
                dataset = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(Y).to(device))
                dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)
    
                # Create a list to store losses for this combination of seq_len and batch_size
                current_loss_list = []
    
                for epoch in range(EPOCHS):
                    for xx, yy in dataloader:
                        optimizer.zero_grad()
                        out = model(xx)
                        loss = criterion(out, yy)
                        loss.backward()
                        optimizer.step()
                    scheduler.step()
                    current_loss_list.append(loss.item())
                    # print(f"Epoch {epoch + 1}/{EPOCHS}: Loss={loss}")
    
                end_time = time.time()
                
                exec_time = end_time - start_time
                execution_time.append(exec_time)
                print(f"Completed Seq Len {seq_len[i]}, Batch Size {BATCH_SIZE},Epoch Size {EPOCHS},Learning Rate {LEARNING_RATE}, in {end_time - start_time:.2f} seconds")
    
                # Save the model
                model_save_dir = "saved_models"
                datetime_model_save_dir = os.path.join(model_save_dir, f"{new_file_number}_{formatted_datetime}")
                os.makedirs(datetime_model_save_dir, exist_ok=True)
                model_save_path = os.path.join(datetime_model_save_dir, f"{run_number}_model_seq_len_{seq_len[i]}_batch_{BATCH_SIZE}_epochs_{EPOCHS}_learning_rate_{LEARNING_RATE}.pth")
                torch.save(model.state_dict(), model_save_path)

                
                # Serialize the model's state_dict
                state_dict_binary = pickle.dumps(model.state_dict())
                # Encode the binary data into base64 to store in a text format
                state_dict_base64 = base64.b64encode(state_dict_binary).decode('utf-8')
                    
                
                
                # Prediction Loop
                model.eval()
                x_copy=np.copy(x)
                predictions = []
                
                for ff in range(len(forcast)):
                    xxx = x_copy[-seq_len[i]:]
                    yyy = model(torch.Tensor(xxx).reshape((1, seq_len[i], 1)).to(device))
                    x_copy= np.concatenate((x_copy, yyy.detach().cpu().numpy().reshape(1,)))
                    predictions.append(yyy.detach().cpu().numpy().flatten())
    

                predictions = np.array(predictions).flatten()
                rmse = np.sqrt(np.mean((predictions - forcast)**2))
                print(f"Completed Seq Len {seq_len[i]}, Batch Size {BATCH_SIZE},Epoch Size {EPOCHS},Learning Rate {LEARNING_RATE}, in {end_time - start_time:.2f} seconds, RMSE: {rmse}")

                run_number=run_number+1

                 # Append results to the results list
                results.append({
                    'seq_len': seq_len[i],
                    'batch_size': BATCH_SIZE,
                    'epochs': EPOCHS,
                    'learning_rate':LEARNING_RATE,
                    'execution_time': exec_time,
                    'RMSE':rmse,
                    'Predictions': str([float(value) for value in predictions]), 
                    'Loss':current_loss_list})

                models_csv.append({
                    'seq_len': seq_len[i],
                    'batch_size': BATCH_SIZE,
                    'epochs': EPOCHS,
                    'learning_rate':LEARNING_RATE,
                    'execution_time': exec_time,
                    'RMSE':rmse,
                    'Predictions':  str([float(value) for value in predictions]),
                    'Loss':current_loss_list,
                    'model_parameters': state_dict_base64,
                })

In [ ]:
#Final Results
results_df = pd.DataFrame(results)
models_csv_df=pd.DataFrame(models_csv)


### AFTER TRAINING

In [ ]:
results_df['Predictions'] = results_df['Predictions'].apply(lambda x: list(map(float, x.strip('[]').split())))
results_df

In [ ]:
#FINDING THE LOWEST RMSE
min_rmse_row = results_df.nsmallest(1, 'RMSE')
min_rmse_row
